In [ ]:
import faiss
import json
from sentence_transformers import SentenceTransformer

# Load FAISS index and metadata
index_file = "vector_index.faiss"  # Path to the saved FAISS index file
metadata_file = "metadata.json"  # Path to the metadata JSON file

# Load the FAISS index into memory
index = faiss.read_index(index_file)

# Load the metadata that maps embeddings to text and categories
with open(metadata_file, 'r', encoding='utf-8') as f:
    metadata = json.load(f)

# Load the embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')  # Ensure the same model is used as during embedding creation

# Query FAISS index
def query_faiss(question, top_k=5):
    """
    Searches the FAISS index for the closest matches to a query.

    Parameters:
        question (str): The user's query in natural language.
        top_k (int): Number of closest matches to retrieve.
    
    Returns:
        list: A list of results containing text and category information.
    """
    # Generate an embedding for the query using the same model
    question_embedding = model.encode([question])

    # Search the FAISS index for the top_k closest embeddings
    distances, indices = index.search(question_embedding, top_k)

    # Retrieve the matching results from metadata
    results = []
    for idx in indices[0]:
        if idx < len(metadata):  # Ensure the index is within metadata bounds
            results.append(metadata[idx])
    return results

# Example queries to test the FAISS search
queries = [
    "What attractions are available at Jewel Changi Airport?",
    "Is there a free tour at Changi Airport?",
    "Tell me about parking options at Changi Airport."
]

# Execute each query and print the results
for query in queries:
    print(f"\nQuery: {query}")
    results = query_faiss(query)  # Get results for the query
    for result in results:
        print(f"- {result['text']} (Category: {result['category']})")
